# Build SPM covariate table

In [27]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1) Paths and column names
Adjust these to your files/sheet names if needed.

In [28]:
base = "C:/Users/juhoffmann/Desktop/SubliminalVideoPriming/data"
cov_path = f"{base}/demographics/all.xlsx"
cov_sheet = 0

spm_path = f"{base}/mri/SPM_mat_subject_row.xlsx"
spm_sheet = 0

rt_path = f"{base}/formatted/reaction_times_long_format.csv"

cov = pd.read_excel(cov_path, sheet_name=cov_sheet)
spm = pd.read_excel(spm_path, sheet_name=spm_sheet)
rt = pd.read_csv(rt_path)
rt.rename(columns={"Participant": "ID", "ReactionTime": "rt"}, inplace=True)

cov.head()
spm.head()

,ID,contrast
0,sub-004,con_0001.nii'
1,sub-006,con_0001.nii'
2,sub-010,con_0001.nii'
3,sub-011,con_0001.nii'
4,sub-014,con_0001.nii'


In [31]:
def normalize_id(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    x = x.replace("Sub-", "sub-").replace("SUB-", "sub-").replace("sub_", "sub-")
    digits = "".join([c for c in x if c.isdigit()])
    return f"sub-{digits.zfill(3)}" if digits else x

cov = cov.replace({"na": np.nan, "NA": np.nan, "": np.nan})
cov["ID"] = cov["ID"].apply(normalize_id)
rt["ID"] = rt["ID"].apply(normalize_id)
spm["ID"] = spm["ID"].apply(normalize_id)   
#cov.head()

In [ ]:
# Covariate columns from cov_sheet
gender_col = "Gender_f1_m2"
group_col = "Group_MDD1_HC2"
med_col = "Antidepressants"
bdi_col = "BDI-II"
ham_col = "Hamilton Score"
rt_col = "rt"
spm_id_col = "ID"
cov_id_col = "ID"
rt_id_col = "ID"    

### Build analysis covariates

In [33]:
# Recode gender and group if present
if gender_col in cov.columns:
    cov["Gender_f1_m2"] = cov[gender_col].map({1: 0, 2: 1})

if group_col in cov.columns:
    cov["Group_MDD1_HC2"] = cov[group_col].map({1: 0, 2: 1})

if med_col in cov.columns:
    cov["Antidepressants"] = cov[med_col].fillna(0)

# Center continuous covariates
if bdi_col in cov.columns:
    cov["BDI_centered"] = pd.to_numeric(cov[bdi_col], errors="coerce") - pd.to_numeric(cov[bdi_col], errors="coerce").mean()
if ham_col in cov.columns:
    cov["Hamilton_centered"] = pd.to_numeric(cov[ham_col], errors="coerce") - pd.to_numeric(cov[ham_col], errors="coerce").mean()

cov.head()

,EmoCon_ID,ID,Antidepressants,Video Mask done,Gender_x,Alter,Gender_f1_m2,Group_MDD1_HC2,BVAQ,BDI-II,...,CERQ_Refokussierung auf Planung,CERQ_Positive Neubewertung,CERQ_Relativieren,CERQ_Katastrophisierung,CERQ_Andere Beschuldigen,Antidepressants.1,Medication,AD_Classes,BDI_centered,Hamilton_centered
0,EmoCon006,sub-004,0.0,1,male,29,1,1,56,6,...,17,13,15,7,7,NaN,NaN,NaN,-8.504274,NaN
1,EmoCon007,sub-006,0.0,1,female,23,0,1,47,3,...,14,16,17,7,10,NaN,NaN,NaN,-11.504274,NaN
2,EmoCon001,sub-007,1.0,1,male,23,1,0,52,26,...,8,4,7,15,4,1.0,morning: Citalopram 30mg,['SSRI'],11.495726,0.45614
3,EmoCon017,sub-008,1.0,1,female,19,0,0,58,35,...,13,11,5,9,7,1.0,morning: Escitalopram 10mg evening: Quetiapin...,['SSRI'],20.495726,2.45614
4,EmoCon015,sub-009,1.0,1,male,27,1,0,59,14,...,12,5,4,14,7,1.0,morning: Elontril 300mg,['NDRI'],-0.504274,1.45614


### Merge into SPM order and export

In [ ]:
merged = spm[[spm_id_col]].merge(cov, left_on=spm_id_col, right_on=cov_id_col, how="left")

# Select final columns to export
final_cols = [
    spm_id_col,
    "Group_MDD1_HC2",
    "Gender_f1_m2",
    "Antidepressants",
    "BDI_centered",
    "Hamilton_centered"
]
final_cols = [c for c in final_cols if c in merged.columns]

cov_table = merged[final_cols]
cov_table.head()

,ID,Group_MDD1_HC2,Gender_f1_m2,Antidepressants,BDI_centered,Hamilton_centered
0,sub-004,1.0,1.0,0.0,-8.504274,NaN
1,sub-006,1.0,0.0,0.0,-11.504274,NaN
2,sub-010,1.0,0.0,0.0,-14.504274,NaN
3,sub-011,1.0,1.0,0.0,-6.504274,NaN
4,sub-014,1.0,1.0,0.0,-14.504274,NaN


In [26]:
out_path = Path(base) / "mri/regressors.csv"
cov_table.to_csv(out_path, index=False)
print("Wrote:", out_path)

Wrote: C:\Users\juhoffmann\Desktop\SubliminalVideoPriming\data\mri\regressors.csv
